In [122]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.metrics import roc_auc_score, f1_score
from sklearn.model_selection import train_test_split, GridSearchCV

# read data

In [5]:
df_train = pd.read_csv('../datasets/train.csv')
df_test = pd.read_csv('../datasets/test.csv')

In [6]:
def convert_passengerID_2_index(df: pd.DataFrame):

    df = df.set_index("PassengerId")
    return df

In [7]:
df_train_cleansed = (
    df_train.pipe(convert_passengerID_2_index)
)

In [ ]:
df_test_cleansed = (
    df_test.pipe(convert_passengerID_2_index)
)

# split data into X and y

In [17]:
def split_data_2_x_y(df:pd.DataFrame):

    columns = df.columns 
    X_columns = [i for i in columns if i != 'Transported']
    y_column = 'Transported' if 'Transported' in columns else None 

    if y_column:
        return df[X_columns], df[y_column]
    else:
        return df[X_columns], None

In [48]:
X, y = split_data_2_x_y(df_train_cleansed)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [49]:
X_test, _ = split_data_2_x_y(df_test_cleansed)

In [50]:
X_train.head()

,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name
PassengerId,,,,,,,,,,,,
2513_01,Earth,False,NaN,TRAPPIST-1e,28.0,False,0.0,55.0,0.0,656.0,0.0,Loree Mathison
2774_02,Earth,False,F/575/P,TRAPPIST-1e,17.0,False,0.0,1195.0,31.0,0.0,0.0,Crisey Mcbriddley
8862_04,Europa,True,C/329/S,55 Cancri e,28.0,False,0.0,0.0,0.0,0.0,0.0,Alramix Myling
8736_02,Mars,False,F/1800/P,TRAPPIST-1e,20.0,False,NaN,2.0,289.0,976.0,0.0,Tros Pota
0539_02,Europa,True,C/18/P,55 Cancri e,36.0,False,0.0,0.0,0.0,0.0,0.0,Achyon Nalanet


# data pipeline

In [51]:
num_pipeline = Pipeline([
    ("Fill N/A value", SimpleImputer(strategy="median")),
    ("standardize", StandardScaler())
])

cate_pipeline = Pipeline([
    ("Fill N/A value", SimpleImputer(strategy="most_frequent")),
    ("One Hot Encoder", OneHotEncoder(handle_unknown="ignore"))
])

In [33]:
# num_attribs = X_train.select_dtypes(include="float").columns
# cat_attribs = X_train.select_dtypes(include="object").columns

In [52]:
preprocessing = ColumnTransformer([
    ("num", num_pipeline, make_column_selector(dtype_include=np.number)),
    ("cat", cate_pipeline, make_column_selector(dtype_include=object))
])

In [41]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8693 entries, 0001_01 to 9280_02
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   HomePlanet    8492 non-null   object 
 1   CryoSleep     8476 non-null   object 
 2   Cabin         8494 non-null   object 
 3   Destination   8511 non-null   object 
 4   Age           8514 non-null   float64
 5   VIP           8490 non-null   object 
 6   RoomService   8512 non-null   float64
 7   FoodCourt     8510 non-null   float64
 8   ShoppingMall  8485 non-null   float64
 9   Spa           8510 non-null   float64
 10  VRDeck        8505 non-null   float64
 11  Name          8493 non-null   object 
dtypes: float64(6), object(6)
memory usage: 882.9+ KB


# Train the model

In [53]:
rnd_clf = make_pipeline(preprocessing, RandomForestClassifier(random_state=42))
rnd_clf.fit(X_train, y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('Fill N/A '
                                                                   'value',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('standardize',
                                                                   StandardScaler())]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x28d3d76d0>),
                                                 ('cat',
                                                  Pipeline(steps=[('Fill N/A '
                                                                   'value',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('One Hot '
                                                                   'Encoder',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x28d3d4520>)])),
                ('randomforestclassifier',
                 RandomForestClassifier(random_state=42))])

# Tunning the model

In [132]:
full_pipeline = Pipeline([
    ("preprocessing", preprocessing),
    ("random_forest", RandomForestClassifier(random_state=42))
])

# param_grid = [
#     {'preprocessing__geo__n_clusters': [5, 8, 10],
#     'random_forest__max_features': [4, 6, 8]},
#     {'preprocessing__geo__n_clusters': [10, 15],
#      'random_forest__max_features': [6, 8, 10]}
# ]
param_grid = [
    {
    'random_forest__max_features': [6, 8, 10],
    'random_forest__n_estimators': [50, 100, 200, 500],
    'random_forest__max_depth': [5, 10, None],
    'random_forest__min_samples_split': [2, 5, 10],
    'random_forest__min_samples_leaf': [1, 2, 4],
    # 'random_forest__max_features': ['auto', 'sqrt']
    }
]
grid_search = GridSearchCV(full_pipeline, param_grid, cv=3,
                           scoring="f1", #"roc_auc", 
                           n_jobs=-1)
grid_search.fit(X_train, y_train)

/Users/liuyifu/miniconda3/envs/sklearn-env/lib/python3.10/site-packages/joblib/externals/loky/process_executor.py:702: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('preprocessing',
                                        ColumnTransformer(transformers=[('num',
                                                                         Pipeline(steps=[('Fill '
                                                                                          'N/A '
                                                                                          'value',
                                                                                          SimpleImputer(strategy='median')),
                                                                                         ('standardize',
                                                                                          StandardScaler())]),
                                                                         <sklearn.compose._column_transformer.make_column_selector object at 0x28d3d76d0>),
                                                                        ('cat',
                                                                         Pipeline(steps=[('Fill '
                                                                                          'N/A '
                                                                                          'value',
                                                                                          SimpleImputer(strategy=...
                                                                         <sklearn.compose._column_transformer.make_column_selector object at 0x28d3d4520>)])),
                                       ('random_forest',
                                        RandomForestClassifier(random_state=42))]),
             n_jobs=-1,
             param_grid=[{'random_forest__max_depth': [5, 10, None],
                          'random_forest__max_features': [6, 8, 10],
                          'random_forest__min_samples_leaf': [1, 2, 4],
                          'random_forest__min_samples_split': [2, 5, 10],
                          'random_forest__n_estimators': [50, 100, 200, 500]}],
             scoring='f1')

In [133]:
print('Best hyperparameters:', grid_search.best_params_)
print('Best score:', grid_search.best_score_)

Best hyperparameters: {'random_forest__max_depth': None, 'random_forest__max_features': 6, 'random_forest__min_samples_leaf': 1, 'random_forest__min_samples_split': 5, 'random_forest__n_estimators': 200}
Best score: 0.7885692673552747


In [134]:
cv_res = pd.DataFrame(grid_search.cv_results_)
cv_res.sort_values(by='mean_test_score', ascending=False, inplace=True)
cv_res.head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_random_forest__max_depth,param_random_forest__max_features,param_random_forest__min_samples_leaf,param_random_forest__min_samples_split,param_random_forest__n_estimators,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
222,2.679770,0.035974,0.980709,0.063563,None,6,1,5,200,"{'random_forest__max_depth': None, 'random_for...",0.774814,0.793679,0.797215,0.788569,0.009833,1
223,6.547056,0.103261,2.257376,0.067186,None,6,1,5,500,"{'random_forest__max_depth': None, 'random_for...",0.773710,0.791776,0.799305,0.788264,0.010740,2
219,7.852763,0.110182,2.302956,0.079419,None,6,1,2,500,"{'random_forest__max_depth': None, 'random_for...",0.774108,0.788462,0.797562,0.786711,0.009655,3
218,3.172699,0.064281,0.987607,0.037717,None,6,1,2,200,"{'random_forest__max_depth': None, 'random_for...",0.774307,0.784998,0.797557,0.785621,0.009502,4
258,2.415997,0.048797,0.808029,0.016920,None,8,1,5,200,"{'random_forest__max_depth': None, 'random_for...",0.780702,0.780702,0.794771,0.785392,0.006632,5


# Evaluate the performance

## Evaluate the performance on the training set

In [125]:
y_proba = rnd_clf.predict_proba(X_train)[:, 1]
y_pred = rnd_clf.predict(X_train)
auc_roc = roc_auc_score(y_train, y_proba)
f1 = f1_score(y_train, y_pred)

# Print the AUC-ROC score
print('AUC-ROC score:', auc_roc)
print('F1 score:', f1)

AUC-ROC score: 1.0
F1 score: 1.0


In [135]:
y_proba = grid_search.predict_proba(X_train)[:, 1]
y_pred = grid_search.predict(X_train)
auc_roc = roc_auc_score(y_train, y_proba)
f1 = f1_score(y_train, y_pred)

# Print the AUC-ROC score
print('AUC-ROC score:', auc_roc)
print('F1 score:', f1)

AUC-ROC score: 0.9999998345603441
F1 score: 0.9997143673236218


## Evaluate the performance on the validation set

In [129]:
# randome forest tree
y_proba = rnd_clf.predict_proba(X_val)[:, 1]
y_pred = rnd_clf.predict(X_val)
auc_roc = roc_auc_score(y_val, y_proba)
f1 = f1_score(y_val, y_pred)

# Print the AUC-ROC score
print('AUC-ROC score:', auc_roc)
print('F1 score:', f1)

AUC-ROC score: 0.8549780543363521
F1 score: 0.7871259175607002


In [136]:
# gridsearchCV
y_proba = grid_search.predict_proba(X_val)[:, 1]
y_pred = grid_search.predict(X_val)
auc_roc = roc_auc_score(y_val, y_proba)
f1 = f1_score(y_val, y_pred)

# Print the AUC-ROC score
print('AUC-ROC score:', auc_roc)
print('F1 score:', f1)

AUC-ROC score: 0.859895655578749
F1 score: 0.7934165720771851


# output data

In [103]:
y_test = rnd_clf.predict(X_test)
output = pd.DataFrame(y_test, index=X_test.index, columns=['Transported'])
output.to_csv('submission.csv')

In [131]:
y_test = grid_search.predict(X_test)
output = pd.DataFrame(y_test, index=X_test.index, columns=['Transported'])
output.to_csv('submission.csv')